In [1]:
import keras
from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

#Data augmentation; flipping images both vertically and horizontally
from keras.preprocessing.image import ImageDataGenerator

#For Tensorboard
import datetime
from keras.callbacks import TensorBoard

import numpy as np

Using TensorFlow backend.


In [2]:
def get_timestamp():
    return '{:%Y-%m-%d-%H-%M-%S}'.format(datetime.datetime.now())

def get_tensorboard(name=None):
    name = name or 'Graph'
    return TensorBoard(
        log_dir='./{0}/{1}'.format(name, get_timestamp()), histogram_freq=0,
        write_graph=True, write_images=False)

In [3]:
# the data, shuffled and split between train and test sets
def load_data(dataset):
    # Load
    (x_train, y_train), (x_test, y_test) = dataset.load_data()
    # Reshape X
    img_rows = x_train.shape[1]
    img_cols = x_train.shape[2]
    n_channels = x_train.shape[3] if len(x_train.shape) == 4 else 1
    if K.image_data_format() == 'channels_first':
        s = [n_channels, img_rows, img_cols]
    else:
        s = [img_rows, img_cols, n_channels]
    x_train = x_train.reshape(x_train.shape[0], *s)
    x_test = x_test.reshape(x_test.shape[0], *s)
    # Rescale X
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    # Reshape y
    num_classes = len(np.unique(y_train))
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    return (x_train, y_train), (x_test, y_test), img_rows, img_cols

In [4]:
(x_train, y_train), (x_test, y_test), img_rows, img_cols = load_data(cifar10)

In [5]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [6]:
#Data augmentation
ImageDataGenerator(horizontal_flip=True, vertical_flip=False, )

In [7]:
linear = Sequential()
linear.add(Flatten(input_shape=x_train.shape[1:]))
linear.add(Dense(units=10, activation='softmax'))

In [8]:
small_cnn = Sequential()

small_cnn.add(Conv2D(32, (2, 2), activation='relu', padding = 'same', input_shape=x_train.shape[1:]))
small_cnn.add(Conv2D(32, (2, 2), activation='relu', padding = 'same'))
small_cnn.add(MaxPooling2D(pool_size=(3, 3)))
small_cnn.add(Dropout(0.3))

small_cnn.add(Conv2D(64, (2, 2), activation='relu', padding = 'same'))
small_cnn.add(Conv2D(64, (2, 2), activation='relu', padding = 'same'))
small_cnn.add(MaxPooling2D(pool_size=(3, 3)))
small_cnn.add(Dropout(0.3))

'''
small_cnn.add(Conv2D(128, (2, 2), activation='relu', padding = 'same'))
small_cnn.add(Conv2D(128, (2, 2), activation='relu', padding = 'same'))
small_cnn.add(MaxPooling2D(pool_size=(3, 3)))
small_cnn.add(Dropout(0.3))
'''

small_cnn.add(Flatten())

small_cnn.add(Dense(units=64, activation='relu'))
small_cnn.add(Dense(units=10, activation='softmax'))


In [9]:
#model = linear
model = small_cnn

In [10]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [11]:
#6000 training samples
n_train    = 6000
batch_size = 128
epochs     = 60

#Tensorboard function
tb = get_tensorboard()

model.fit(x_train[:n_train], y_train[:n_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test), 
          callbacks=[tb])

Train on 6000 samples, validate on 10000 samples
Epoch 1/60
6000/6000 [==============================] - 24s - loss: 2.2944 - acc: 0.1173 - val_loss: 2.2585 - val_acc: 0.1746

KeyboardInterrupt: 

In [ ]:
_, acc = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', acc)